# Generate Synthetic Images

This notebook generates synthetic social movement images using Google Gemini API, combining data from Atropia, World Bank demographics, and visual references.

*Workshop*: AI/ML Pipeline - Synthetic Data Generation; January 23, 2026  
*Platform*: CyVerse Jupyter Lab PyTorch GPU

## Pipeline Overview

1. Load configuration and source data
2. Initialize API client with rate limiting
3. Build prompts from combined data sources
4. Generate images in batches with checkpoints
5. Save images and metadata
6. Summarize results

## Setup and Imports

In [1]:
import sys
from pathlib import Path
import time
from datetime import datetime
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

#text to image model
import os
import torch
import logging
from diffusers import FluxPipeline
from transformers import BitsAndBytesConfig
from diffusers import FluxTransformer2DModel

print("imported things")

# Add parent directory to path
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

print("bringing in local modules")
# brining in more modeules because of image generation process
# modules are found in the DataCollection/src folder
# from src import config, gemini_client, data_loader, prompt_builder, output_handler
from src import config, data_loader, prompt_builder, output_handler

print("All modules imported successfully")
print(f"Working directory: {Path.cwd()}")

# Load configuration
# DataCollection/src/config.py ... def load_confi()
# using "generation_config.yaml" for setup
cfg = config.load_config()



KeyboardInterrupt: 

### 1. Load Configuration

Review and adjust generation parameters if needed.
`DataCollection/config/generation_config.yaml`

In [2]:

# see what was loaded from the "generation_config.yaml" 
print("Current Configuration:")
print("=" * 80)
print(f"\nImages to generate: {cfg.generation['num_images']}")
print(f"Batch size: {cfg.generation['batch_size']}")
print(f"Resolution: {cfg.generation['resolution']}")
print(f"Model: {cfg.generation['model']}")
# print(f"If model is gemini-2.5-flash-image, it's AKA Nano Banana")
print(f"\nPrompt style: {cfg.prompts['style']}")
print(f"Prompt complexity: {cfg.prompts['complexity']}")
print(f"\nRate limit: {cfg.rate_limiting['requests_per_minute']} requests/minute")

print("\n" + "=" * 80)

Current Configuration:

Images to generate: 50
Batch size: 10
Resolution: 1K
Model: black-forest-labs/FLUX.1-schnell

Prompt style: realistic
Prompt complexity: medium

Rate limit: 10 requests/minute



### 2. Load Source Data

Load data from all three sources: Atropia, World Bank, and social media references.

In [3]:
print("Loading source data.\n")

data_dir = cfg.get_data_path('raw')

# Initialize data loaders
atropia_loader = data_loader.AtropiaDataLoader(data_dir=data_dir)
worldbank_loader = data_loader.WorldBankDataLoader(data_dir=data_dir)
socialmedia_loader = data_loader.SocialMediaDataLoader(data_dir=data_dir)

# Load datasets
atropia_data = atropia_loader.load_data()
print(f"Loaded {len(atropia_data)} Atropia samples")

worldbank_data = worldbank_loader.load_data()
print(f"Loaded {len(worldbank_data)} World Bank profiles")

socialmedia_data = socialmedia_loader.load_descriptions()
print(f"Loaded {len(socialmedia_data)} visual references")

# Initialize combiner
combiner = data_loader.DataCombiner(
    atropia_loader=atropia_loader,
    worldbank_loader=worldbank_loader,
    socialmedia_loader=socialmedia_loader
)

print("\nAll source data loaded and ready")

2026-01-22 15:57:36,106 - src.data_loader - INFO - Loading Atropia data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\atropia_samples.json
2026-01-22 15:57:36,106 - src.data_loader - INFO - Loading World Bank data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\worldbank_demographics.csv
2026-01-22 15:57:36,150 - src.data_loader - INFO - Loading visual descriptions from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\imagepath_labels_descp.json


Loading source data.

Loaded 100 Atropia samples
Loaded 50 World Bank profiles
Loaded 16567 visual references

All source data loaded and ready


## 3. Build Prompts

Generate prompts by combining data from all sources.

In [4]:
print("Building prompts\n")

# Initialize prompt builder
# DataCollection/src/prompt_builder.py > class PromptBuilder
builder = prompt_builder.PromptBuilder(
    style=cfg.prompts['style'],
    complexity=cfg.prompts['complexity'],
    include_temporal=cfg.prompts['include_temporal_context'],
    include_demographics=cfg.prompts['include_demographics'],
    themes=cfg.prompts['themes'],
    settings=cfg.prompts['settings']
)

# Generate combined data samples
num_images = cfg.generation['num_images'] #50
#combiner comes from data_loader.py > class DataCombiner
combined_samples = combiner.sample_combined(n=num_images)

# Build prompts
prompts_data = builder.build_batch_prompts(combined_samples)

print(f"  Built {len(prompts_data)} prompts")
print(f"  Style: {cfg.prompts['style']}")
print(f"  Complexity: {cfg.prompts['complexity']}")

2026-01-22 15:57:40,427 - src.data_loader - INFO - Loading Atropia data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\atropia_samples.json
2026-01-22 15:57:40,435 - src.data_loader - INFO - Loading World Bank data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\worldbank_demographics.csv
2026-01-22 15:57:40,479 - src.data_loader - INFO - Loading visual descriptions from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\imagepath_labels_descp.json
2026-01-22 15:57:40,544 - src.data_loader - INFO - Loading Atropia data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\atropia_samples.json
2026-01-22 15:57:40,544 - src.data_loader - INFO - Loading World Bank data fr

Building prompts



2026-01-22 15:57:40,662 - src.data_loader - INFO - Loading Atropia data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\atropia_samples.json
2026-01-22 15:57:40,662 - src.data_loader - INFO - Loading World Bank data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\worldbank_demographics.csv
2026-01-22 15:57:40,669 - src.data_loader - INFO - Loading visual descriptions from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\imagepath_labels_descp.json
2026-01-22 15:57:40,745 - src.data_loader - INFO - Loading Atropia data from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\raw\atropia_samples.json
2026-01-22 15:57:40,745 - src.data_loader - INFO - Loading World Bank data fr

  Built 50 prompts
  Style: realistic
  Complexity: medium


### Preview Sample Prompts

Let's review a few prompts before generation.

In [5]:
print("Sample Prompts:")
print("=" * 80)

for i in range(min(3, len(prompts_data))):
    prompt_info = prompts_data[i]
    print(f"\nPrompt {i+1}:")
    print(f"  {prompt_info['prompt'][:200]}...")
    print(f"\n  Source - Theme: {prompt_info['source_data']['atropia']['theme']}")
    print(f"  Source - Demographics: Age {prompt_info['source_data']['demographics']['age_group']}, "
          f"{prompt_info['source_data']['demographics']['occupation']}")
    print("-" * 80)

Sample Prompts:

Prompt 1:
  photorealistic, high detail, natural lighting. A scene depicting protests with a digital image of a woman with her hands covering her face and text in spanish, supporting a social movement.. Diverse c...

  Source - Theme: protests
  Source - Demographics: Age 45-54, service
--------------------------------------------------------------------------------

Prompt 2:
  photorealistic, high detail, natural lighting. A scene depicting elections with an image of a man and a woman standing side by side with text overlay in spanish.. Diverse crowd featuring people in the...

  Source - Theme: elections
  Source - Demographics: Age 35-44, student
--------------------------------------------------------------------------------

Prompt 3:
  photorealistic, high detail, natural lighting. A scene depicting government response with a wall of skulls, likely a display of ancient artifacts, with a griddy logo in the bottom-right corner.. Diver...

  Source - Theme: governm

### Bring in Model to generate images
FLUX.1 [schnell] pulled from HuggingFace. More information on the model can be found here:
- https://huggingface.co/black-forest-labs/FLUX.1-schnell
- https://bfl.ai/blog/24-08-01-bfl

*Repository:* Black Forest Labs. (2024). FLUX [Computer software]. GitHub. https://github.com/black-forest-labs/flux
*Paper:* Black Forest Labs, Batifol, S., Blattmann, A., Boesel, F., Consul, S., Diagne, C., Dockhorn, T., English, J., English, Z., Esser, P., Kulal, S., Lacey, K., Levi, Y., Li, C., Lorenz, D., Müller, J., Podell, D., Rombach, R., Saini, H., . . . Smith, L. (2025). FLUX.1 Kontext: Flow matching for in-context image generation and editing in latent space. arXiv. https://arxiv.org/abs/2506.15742


In [ ]:

def run_flux(GPU):
    if GPU:

        ###### for GPU
        # Load the FLUX.1 [schnell] model small version 
        # i.e. optimized because there are billions of parameters here and takes up like > 16 RAM
        # Configure the model to load in 4-bit mode (The "Smaller" Version)
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        # Load the Transformer specifically with quantization
        transformer = FluxTransformer2DModel.from_pretrained(
            "black-forest-labs/FLUX.1-schnell",
            subfolder="transformer",
            quantization_config=quantization_config,
            torch_dtype=torch.bfloat16
        )
        # use bfloat16 for efficiency as recommended for Flux in huggingface
        # print(f"Loading Model: {cfg.generation['model']}")
        pipe = FluxPipeline.from_pretrained(
            cfg.generation['model'], # "black-forest-labs/FLUX.1-schnell"
            transformer=transformer,
            torch_dtype=torch.bfloat16
        )
        # Optimization: Offload to CPU to save VRAM (Remove if you have 24GB+ VRAM)
        pipe.enable_model_cpu_offload()
        ######

    else:


        # should take 10-20 minutes to run without a GPU
        #using float32 to save space
        print(f"Loading bits and bytes optimized Model: {cfg.generation['model']}")
        pipe = FluxPipeline.from_pretrained(
            cfg.generation['model'], # "black-forest-labs/FLUX.1-schnell"
            # torch_dtype=torch.float32 #depreciated
            dtype=torch.float32
        )

# Initialize Output Handler & Load Model 
print("Initializing Output Handler and FLUX\n")

pipe = run_flux(GPU = False)
print("\nModel loaded and ready for generation.")

Initializing Output Handler and FLUX

Loading Model: black-forest-labs/FLUX.1-schnell


Keyword arguments {'dtype': torch.float32} are not expected by FluxPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [ ]:
# Based on the output handler module,
# This automatically creates folders in DataCollection/data/generated: images/, metadata/, logs/
handler = output_handler.OutputHandler(
    output_dir=cfg.get_output_path(),  # Uses path from generation_config.yaml
    image_format=cfg.output.get('format', 'png'),
    export_csv=True,
    date_organized=True
)